In [1]:
import scanpy as sc
import pandas as pd
import pickle
import numpy as np
import graph_tool.all as gt
%matplotlib inline
%pylab

/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/draw/cairo_draw.py:39: RuntimeWarning: Error importing cairo. Graph drawing will not work.
  warnings.warn(msg, RuntimeWarning)
/usr/lib/python3/dist-packages/graph_tool/all.py:40: RuntimeWarning: Error importing draw module, proceeding nevertheless: /usr/lib/python3/dist-packages/cairo/_cairo.cpython-37m-x86_64-linux-gnu.so: undefined symbol: cairo_tee_surface_index
  warnings.warn(msg, RuntimeWarning)


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [2]:
import anndata
adata=sc.read('adata_10x/adata_sc_10x.h5ad')
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
df_significativo= pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20) #### df con i 20 geni significativi per gruppo
cell_annotation= pd.read_table('../sc_mixology/data/csv/sc_10x_5cl.metadata.csv.gz', compression="gzip", sep=",", index_col=0)
adata.obs.loc[:, "cell_line"] = cell_annotation.loc[adata.obs_names, "cell_line"]

In [22]:
g = gt.Graph(directed=False)
g

<Graph object, undirected, with 0 vertices and 0 edges at 0x7fa388a85d10>

In [4]:
adata.uns['neighbors']

{'connectivities': <860x860 sparse matrix of type '<class 'numpy.float64'>'
 	with 12014 stored elements in Compressed Sparse Row format>,
 'distances': <860x860 sparse matrix of type '<class 'numpy.float64'>'
 	with 7740 stored elements in Compressed Sparse Row format>,
 'params': {'method': 'umap',
  'metric': 'euclidean',
  'n_neighbors': 10,
  'n_pcs': 40}}

In [28]:
adj=adata.uns['neighbors']['connectivities']
print(adj)

  (0, 3)	1.0
  (0, 4)	0.08516783496033718
  (0, 5)	0.5324798114295779
  (0, 6)	0.6472797497416668
  (0, 10)	0.32859563442624135
  (0, 18)	0.19712161308736198
  (0, 24)	0.25255344699539767
  (0, 30)	0.2108877608071739
  (0, 31)	0.4172287095915445
  (0, 88)	0.21980422215900267
  (0, 839)	0.25499938506173925
  (1, 2)	1.0
  (1, 28)	0.27979636077006603
  (1, 51)	0.250020200318358
  (1, 62)	0.2911784049363471
  (1, 65)	0.2347224583334965
  (1, 93)	0.2003520159061067
  (1, 103)	0.25655547806096646
  (1, 123)	0.5658598576361251
  (1, 302)	0.2434392405319874
  (1, 632)	0.23886128081422467
  (2, 1)	1.0
  (2, 12)	0.2048727199869503
  (2, 23)	0.21191042038173014
  (2, 28)	0.5715436807634252
  :	:
  (857, 375)	0.34751240159787944
  (857, 391)	0.21046271295236876
  (857, 455)	0.2668507409909808
  (857, 464)	0.1883283268858911
  (857, 500)	1.0
  (857, 548)	0.3412814799232153
  (857, 568)	0.40197208983827093
  (858, 421)	0.24239385367230148
  (858, 456)	0.35709284809259373
  (858, 616)	0.2912755513966

In [29]:
edge_weights = g.new_edge_property('double')
print(edge_weights)

<EdgePropertyMap object with value type 'double', for Graph 0x7fa388a85d10, at 0x7fa384a91790>


In [17]:
g.edge_properties['weight'] = edge_weights
g.edge_properties

{'weight': <EdgePropertyMap object with value type 'double', for Graph 0x7fa3db36e050, at 0x7fa38b6b8e90>}

In [18]:
nnz = np.nonzero(np.triu(adj.todense(),1))
nnz

(array([  0,   0,   0, ..., 842, 848, 856]),
 array([  3,   4,   5, ..., 855, 859, 858]))

In [19]:
nedges = len(nnz[0])
nedges

6007

In [20]:
g.add_edge_list(np.hstack([np.transpose(nnz),np.reshape(adj[nnz],(nedges,1))]),eprops=[edge_weights])

In [31]:
g

<Graph object, undirected, with 0 vertices and 0 edges at 0x7fa388a85d10>